In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.bca.co.id/id/informasi/kurs"
headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

tables = soup.find_all("table", class_="m-table-kurs")

table_erat = tables[0]
data_erat = []
for row in table_erat.find_all("tr"):
    cols = row.find_all(["td", "th"])
    text = [col.get_text(strip=True) for col in cols]
    if text:
        data_erat.append(text)

print("\n=== hasil mentah ===")
for r in data_erat:
    print(r)



=== hasil mentah ===
['Mata Uang', 'e-Rate24 Jul 2025 / 17.34WIB', 'TT Counter24 Jul 2025 / 15.26WIB', 'Bank Notes24 Jul 2025 / 15.45WIB']
['Beli', 'Jual', 'Beli', 'Jual', 'Beli', 'Jual']
['USD', '16.250,00', '16.300,00', '16.125,00', '16.425,00', '16.125,00', '16.425,00']
['SGD', '12.710,13', '12.767,59', '12.633,82', '12.878,82', '12.628,00', '12.873,00']
['EUR', '19.084,00', '19.151,34', '18.959,15', '19.362,15', '18.944,00', '19.347,00']
['AUD', '10.724,05', '10.789,26', '10.635,70', '10.899,70', '10.643,00', '10.891,00']
['DKK', '2.539,25', '2.583,36', '2.535,25', '2.608,25', '0,00', '0,00']
['SEK', '1.685,79', '1.727,79', '1.686,80', '1.740,30', '0,00', '0,00']
['CAD', '11.925,00', '11.975,62', '11.827,30', '12.093,30', '11.826,00', '12.096,00']
['CHF', '20.422,42', '20.528,84', '20.298,45', '20.754,45', '20.291,00', '20.719,00']
['NZD', '9.805,14', '9.869,15', '9.708,95', '9.987,95', '9.733,00', '9.965,00']
['GBP', '21.984,88', '22.083,27', '21.841,70', '22.319,70', '21.822,00'

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.bca.co.id/id/informasi/kurs"
headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# cari tabel
table = soup.find("table", class_="m-table-kurs")

data = []
rows = table.find_all("tr")

for row in rows:
    cols = row.find_all("td")
    text = [col.get_text(strip=True) for col in cols]
    if text and len(text) == 7:
        data.append(text)

df = pd.DataFrame(data, columns=[
    "Mata Uang",
    "e-Rate Beli",
    "e-Rate Jual",
    "TT Counter Beli",
    "TT Counter Jual",
    "Bank Notes Beli",
    "Bank Notes Jual"
])

print(df)

# simpan ke CSV
df.to_csv("kurs_bca.csv", index=False, encoding="utf-8-sig")


   Mata Uang e-Rate Beli e-Rate Jual TT Counter Beli TT Counter Jual  \
0        USD   16.250,00   16.300,00       16.125,00       16.425,00   
1        SGD   12.710,13   12.767,59       12.633,82       12.878,82   
2        EUR   19.084,00   19.151,34       18.959,15       19.362,15   
3        AUD   10.724,05   10.787,63       10.635,70       10.899,70   
4        DKK    2.539,33    2.583,03        2.535,25        2.608,25   
5        SEK    1.685,93    1.727,92        1.686,80        1.740,30   
6        CAD   11.925,00   11.975,62       11.827,30       12.093,30   
7        CHF   20.422,42   20.528,84       20.298,45       20.754,45   
8        NZD    9.803,52    9.867,52        9.708,95        9.987,95   
9        GBP   21.984,88   22.076,75       21.841,70       22.319,70   
10       HKD    2.067,00    2.079,44        2.043,60        2.103,20   
11       JPY      110,58      111,48          109,61          112,83   
12       SAR    4.329,46    4.346,80        4.284,60        4.39

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import dash
from dash import dcc, html, dash_table, Input, Output
import plotly.express as px

# URL sumber data kurs
url = "https://www.bca.co.id/id/informasi/kurs"

headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Cari tabel kurs
table = soup.find("table", class_="m-table-kurs")

# Validasi jika tabel ditemukan
if not table:
    raise Exception("Tabel kurs tidak ditemukan. Periksa kembali struktur HTML.")

rows = table.find_all("tr")
data = []

for row in rows:
    cols = row.find_all("td")
    text = [col.get_text(strip=True) for col in cols]
    if text and len(text) == 7:
        data.append(text)

# Buat DataFrame
df = pd.DataFrame(data, columns=[
    "Mata Uang", "e-Rate Beli", "e-Rate Jual",
    "TT Counter Beli", "TT Counter Jual",
    "Bank Notes Beli", "Bank Notes Jual"
])

# Inisialisasi aplikasi Dash
app = dash.Dash(__name__)
app.title = "Kurs BCA"

app.layout = html.Div([
    html.H2("Kurs Mata Uang BCA", style={"textAlign": "center"}),

    html.Div([
        html.Div([
            html.Label("Pilih Mata Uang:"),
            dcc.Dropdown(
                id="currency-dropdown",
                options=[{"label": "Semua", "value": "ALL"}] + 
                        [{"label": currency, "value": currency} for currency in df["Mata Uang"].unique()],
                value="ALL",
                clearable=False,
                style={"width": "250px"}
            )
        ], style={"display": "inline-block", "marginRight": "30px"}),

        html.Div([
            html.Label("Pilih Kategori:"),
            dcc.Dropdown(
                id="category-dropdown",
                options=[
                    {"label": "Semua", "value": "ALL"},
                    {"label": "e-Rate", "value": "ERATE"},
                    {"label": "TT Counter", "value": "TTC"},
                    {"label": "Bank Notes", "value": "BN"}
                ],
                value="ALL",
                clearable=False,
                style={"width": "220px"}
            )
        ], style={"display": "inline-block"}),
    ], style={"textAlign": "center", "marginBottom": "30px"}),

    dash_table.DataTable(
        id="currency-table",
        columns=[{"name": col, "id": col} for col in df.columns],
        data=df.to_dict("records"),
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "center"},
        style_header={"backgroundColor": "#0e76a8", "color": "white"},
    ),

    html.Br(),

    html.Div([
        dcc.Graph(id="currency-graph")
    ])
])

@app.callback(
    Output("currency-table", "data"),
    Output("currency-table", "columns"),
    Output("currency-graph", "figure"),
    Input("currency-dropdown", "value"),
    Input("category-dropdown", "value")
)
def update_output(selected_currency, selected_category):
    col_prefix = None

    if selected_currency == "ALL":
        filtered_df = df.copy()
    else:
        filtered_df = df[df["Mata Uang"] == selected_currency]

    if selected_category == "ERATE":
        cols = ["Mata Uang", "e-Rate Beli", "e-Rate Jual"]
        col_prefix = "e-Rate"
    elif selected_category == "TTC":
        cols = ["Mata Uang", "TT Counter Beli", "TT Counter Jual"]
        col_prefix = "TT Counter"
    elif selected_category == "BN":
        cols = ["Mata Uang", "Bank Notes Beli", "Bank Notes Jual"]
        col_prefix = "Bank Notes"
    else:
        cols = df.columns.tolist()
        col_prefix = None

    table_data = filtered_df[cols].to_dict("records")
    table_columns = [{"name": col, "id": col} for col in cols]

    if col_prefix:
        plot_df = filtered_df[["Mata Uang", f"{col_prefix} Beli", f"{col_prefix} Jual"]].copy()
        plot_df = plot_df.melt(id_vars="Mata Uang", var_name="Tipe", value_name="Kurs")

        fig = px.bar(
            plot_df,
            x="Mata Uang",
            y="Kurs",
            color="Tipe",
            barmode="group",
            title=f"Perbandingan {col_prefix} Beli & Jual",
            log_y=True
        )

        fig.update_layout(
            yaxis=dict(
                showticklabels=False,
                showgrid=False
            )
        )
    else:
        fig = {}

    return table_data, table_columns, fig

# Jalankan aplikasi
if __name__ == "__main__":
    app.run(debug=True)
